<a href="https://colab.research.google.com/github/KingDurian/oss/blob/main/parkinsons_data_fold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:


from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/config.py .
!cp drive/MyDrive/model_dispatcher.py .
import config
import model_dispatcher



import importlib.util


importlib.reload(config)
importlib.reload(model_dispatcher)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<module 'model_dispatcher' from '/content/model_dispatcher.py'>

In [93]:


import pandas as pd
import numpy as np

from sklearn import model_selection
# Column Transformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split


## Read CSV file

In [94]:
data_file = config.SOURCE_FILE

In [95]:
df = pd.read_csv(data_file)

In [96]:
df.shape

(195, 24)

In [97]:
df.head()

,name,MDVP_Fo_Hz_,MDVP_Fhi_Hz_,MDVP_Flo_Hz_,MDVP_Jitter_,MDVP_Jitter_Abs_,MDVP_RAP,MDVP_PPQ,Jitter_DDP,MDVP_Shimmer,...,Shimmer_DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [98]:
df.tail()

,name,MDVP_Fo_Hz_,MDVP_Fhi_Hz_,MDVP_Flo_Hz_,MDVP_Jitter_,MDVP_Jitter_Abs_,MDVP_RAP,MDVP_PPQ,Jitter_DDP,MDVP_Shimmer,...,Shimmer_DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
190,phon_R01_S50_2,174.188,230.978,94.261,0.00459,0.00003,0.00263,0.00259,0.00790,0.04087,...,0.07008,0.02764,19.517,0,0.448439,0.657899,-6.538586,0.121952,2.657476,0.133050
191,phon_R01_S50_3,209.516,253.017,89.488,0.00564,0.00003,0.00331,0.00292,0.00994,0.02751,...,0.04812,0.01810,19.147,0,0.431674,0.683244,-6.195325,0.129303,2.784312,0.168895
192,phon_R01_S50_4,174.688,240.005,74.287,0.01360,0.00008,0.00624,0.00564,0.01873,0.02308,...,0.03804,0.10715,17.883,0,0.407567,0.655683,-6.787197,0.158453,2.679772,0.131728
193,phon_R01_S50_5,198.764,396.961,74.904,0.00740,0.00004,0.00370,0.00390,0.01109,0.02296,...,0.03794,0.07223,19.020,0,0.451221,0.643956,-6.744577,0.207454,2.138608,0.123306
194,phon_R01_S50_6,214.289,260.277,77.973,0.00567,0.00003,0.00295,0.00317,0.00885,0.01884,...,0.03078,0.04398,21.209,0,0.462803,0.664357,-5.724056,0.190667,2.555477,0.148569


## Handle Outliers

In [99]:
numeric_features = list(df.columns)

In [100]:
numeric_features.remove('name')
numeric_features.remove('status')

In [101]:
numeric_features

['MDVP_Fo_Hz_',
 'MDVP_Fhi_Hz_',
 'MDVP_Flo_Hz_',
 'MDVP_Jitter_',
 'MDVP_Jitter_Abs_',
 'MDVP_RAP',
 'MDVP_PPQ',
 'Jitter_DDP',
 'MDVP_Shimmer',
 'MDVP_Shimmer_dB_',
 'Shimmer_APQ3',
 'Shimmer_APQ5',
 'MDVP_APQ',
 'Shimmer_DDA',
 'NHR',
 'HNR',
 'RPDE',
 'DFA',
 'spread1',
 'spread2',
 'D2',
 'PPE']

In [102]:
for feature in numeric_features:
    percent_99 = np.percentile(df[feature], 99)
    percent_1 =  np.percentile(df[feature], 1)
    outliers = sum((df[feature] > percent_99) | (df[feature] < percent_1))

    print(f"{feature}:")
    print(f"  percent_1:\t{percent_1:.3f}")
    print(f"  percent_99:\t{percent_99:.3f}")
    print(f"  min: {df[feature].min():.3f}\tmax: {df[feature].max():.3f}")
    print(f"  outliers: {outliers}")
    
    # set max at 99th percentile
    df.loc[df[feature] > percent_99, feature] = percent_99
    # set min at 1st percentile
    df.loc[df[feature] < percent_1, feature] = percent_1

MDVP_Fo_Hz_:
  percent_1:	94.867
  percent_99:	245.927
  min: 88.333	max: 260.105
  outliers: 4
MDVP_Fhi_Hz_:
  percent_1:	107.390
  percent_99:	586.684
  min: 102.145	max: 592.030
  outliers: 4
MDVP_Flo_Hz_:
  percent_1:	65.780
  percent_99:	232.772
  min: 65.476	max: 239.170
  outliers: 4
MDVP_Jitter_:
  percent_1:	0.002
  percent_99:	0.030
  min: 0.002	max: 0.033
  outliers: 4
MDVP_Jitter_Abs_:
  percent_1:	0.000
  percent_99:	0.000
  min: 0.000	max: 0.000
  outliers: 3
MDVP_RAP:
  percent_1:	0.001
  percent_99:	0.018
  min: 0.001	max: 0.021
  outliers: 4
MDVP_PPQ:
  percent_1:	0.001
  percent_99:	0.016
  min: 0.001	max: 0.020
  outliers: 4
Jitter_DDP:
  percent_1:	0.002
  percent_99:	0.054
  min: 0.002	max: 0.064
  outliers: 4
MDVP_Shimmer:
  percent_1:	0.010
  percent_99:	0.092
  min: 0.010	max: 0.119
  outliers: 4
MDVP_Shimmer_dB_:
  percent_1:	0.089
  percent_99:	0.935
  min: 0.085	max: 1.302
  outliers: 4
Shimmer_APQ3:
  percent_1:	0.005
  percent_99:	0.054
  min: 0.005	max: 0.

## Data Preprocessing

In [103]:

df = df.drop(columns=['name'])

In [104]:
df.columns

Index(['MDVP_Fo_Hz_', 'MDVP_Fhi_Hz_', 'MDVP_Flo_Hz_', 'MDVP_Jitter_',
       'MDVP_Jitter_Abs_', 'MDVP_RAP', 'MDVP_PPQ', 'Jitter_DDP',
       'MDVP_Shimmer', 'MDVP_Shimmer_dB_', 'Shimmer_APQ3', 'Shimmer_APQ5',
       'MDVP_APQ', 'Shimmer_DDA', 'NHR', 'HNR', 'status', 'RPDE', 'DFA',
       'spread1', 'spread2', 'D2', 'PPE'],
      dtype='object')

In [105]:
df.head()

,MDVP_Fo_Hz_,MDVP_Fhi_Hz_,MDVP_Flo_Hz_,MDVP_Jitter_,MDVP_Jitter_Abs_,MDVP_RAP,MDVP_PPQ,Jitter_DDP,MDVP_Shimmer,MDVP_Shimmer_dB_,...,Shimmer_DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,0.426,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,0.626,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,0.482,...,0.08270,0.01309,20.651,1,0.429895,0.823579,-4.443179,0.311173,2.342259,0.332634
3,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,0.517,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,0.584,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [106]:
# Skewed Data Set
df['status'].value_counts()

1    147
0     48
Name: status, dtype: int64

In [107]:
normal_count = df['status'].value_counts()[0]
print(normal_count)

48


## Stratified K-Fold

In [108]:
df["kfold"] = -1

kf = model_selection.StratifiedKFold(n_splits=5)

In [109]:
# iterate through all the folds
# identify all the rows that will be used as validation set during each fold
for fold,(train_idx, validation_idx) in enumerate(kf.split(X=df,y=df['status'])):
    print(fold, train_idx[:5], validation_idx[:5])
    df.loc[validation_idx,'kfold'] = fold

0 [36 37 38 39 40] [0 1 2 3 4]
1 [0 1 2 3 4] [36 37 38 39 40]
2 [0 1 2 3 4] [60 61 62 63 64]
3 [0 1 2 3 4] [113 114 115 116 117]
4 [0 1 2 3 4] [142 143 144 145 146]


In [110]:
df.to_csv(config.TRAINING_FILE,index=False)